In [ ]:
!nvidia-smi

Sun May  7 17:36:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cp /content/drive/MyDrive/DIV2K.rar .

In [ ]:
! apt-get install rar

In [ ]:
!unrar x /content/DIV2K.rar /content

In [ ]:
%pip install -U transformers datasets evaluate

###DIV2K dataset preprocessing


In [ ]:
import os
import random
from PIL import Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from transformers import PoolFormerFeatureExtractor, PoolFormerModel
from transformers import SegformerForSemanticSegmentation, SegformerDecodeHead
from transformers.modeling_outputs import SemanticSegmenterOutput
from datasets import load_dataset, Dataset
from torchsummary import summary
import torch
from typing import Optional, Tuple, Union
from tqdm import tqdm
from copy import deepcopy
from torchvision.transforms import RandomResizedCrop, Compose, ToTensor

In [ ]:

x4dir = ''       #input folder
hrdir = ''       #GT folder
inputlist = os.listdir(x4dir)
random.shuffle(inputlist)
trainlist = inputlist[:3550]
vallist = inputlist[3550:]

my_dict = {'image': [], 'label': []}
for ori_img in trainlist:
    label = Image.open(hrdir + ori_img.replace('x4',''))
    ori_img = Image.open(x4dir + ori_img)
    ori_img = ori_img.resize(label.size, resample=0)
    my_dict['image'].append(ori_img)
    my_dict['label'].append(label)
train_dataset = Dataset.from_dict(my_dict)
del my_dict

my_dict = {'image': [], 'label': []}
for ori_img in vallist:
    label = Image.open(hrdir + ori_img.replace('x4',''))
    ori_img = Image.open(x4dir + ori_img)
    ori_img = ori_img.resize(label.size, resample=0)
    my_dict['image'].append(ori_img)
    my_dict['label'].append(label)
val_dataset = Dataset.from_dict(my_dict)
del my_dict

In [ ]:
def transforms(examples):
    images = [ToTensor()(img.convert("RGB")) for img in examples["image"]]
    labels = [ToTensor()(label.convert("RGB")) for label in examples["label"]]
    cropped = deepcopy([torch.cat([x,y], dim=0) for x, y in zip(images, labels)])
    cropped = [RandomResizedCrop(256)(stack) for stack in cropped]
    examples["pixel_values"] = [stack[:3] for stack in cropped]
    examples["annotation"] = [stack[3:] for stack in cropped]
    del examples["image"], examples["label"], images, labels, cropped
    return examples

In [ ]:
train_dataset = train_dataset.with_transform(transforms)
val_dataset = val_dataset.with_transform(transforms)

##Train

In [ ]:
class Img2ImgDecoder(SegformerDecodeHead):
    def __init__(self, config):
        super().__init__(config)
        self.classifier = torch.nn.Conv2d(config.decoder_hidden_size, 3, kernel_size=1)
        self.RGBnorm = torch.nn.BatchNorm2d(3)

    def forward(self, encoder_hidden_states):
        batch_size = encoder_hidden_states[-1].shape[0]
        h_4, w_4 = encoder_hidden_states[0].size()[2:]

        all_hidden_states = ()
        for encoder_hidden_state, mlp in zip(encoder_hidden_states, self.linear_c):
            if self.config.reshape_last_stage is False and encoder_hidden_state.ndim == 3:
                height = width = int(math.sqrt(encoder_hidden_state.shape[-1]))
                encoder_hidden_state = (
                    encoder_hidden_state.reshape(batch_size, height, width, -1).permute(0, 3, 1, 2).contiguous()
                )

            # unify channel dimension
            height, width = encoder_hidden_state.shape[2], encoder_hidden_state.shape[3]
            encoder_hidden_state = mlp(encoder_hidden_state)
            encoder_hidden_state = encoder_hidden_state.permute(0, 2, 1)
            encoder_hidden_state = encoder_hidden_state.reshape(batch_size, -1, height, width)
            # upsample
            encoder_hidden_state = torch.nn.functional.interpolate(
                encoder_hidden_state, size=(h_4*4, w_4*4), mode="bicubic", align_corners=False
            )
            all_hidden_states += (encoder_hidden_state,)

        hidden_states = self.linear_fuse(torch.cat(all_hidden_states[::-1], dim=1))
        hidden_states = self.batch_norm(hidden_states)
        hidden_states = self.activation(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # logits are of shape (batch_size, RGB, height, width)
        logits = self.classifier(hidden_states)
        logits = self.RGBnorm(logits)
        logits = torch.sigmoid(logits)

        return logits

In [ ]:
Encoder = PoolFormerModel.from_pretrained("sail/poolformer_s36"); Encoder.to('cuda:0')
Decoder = Img2ImgDecoder.from_pretrained("nvidia/mit-b1"); Decoder.to('cuda:0')
#Encoder = PoolFormerModel.from_pretrained("/content/drive/MyDrive/Poolformer_Segformer/checkpoint-1200"); Encoder.to('cuda:0') #
#Decoder = Img2ImgDecoder.from_pretrained("/content/drive/MyDrive/Poolformer_Segformer/checkpoint-1200"); Decoder.to('cuda:0') #
print('Loading from pretrained weights: Done!')

In [ ]:
feature_extractor = PoolFormerFeatureExtractor.from_pretrained("sail/poolformer_s36")

In [ ]:
class Img2Img(SegformerForSemanticSegmentation):
    def __init__(self, config):
        super().__init__(config)
        self.segformer = Encoder
        self.decode_head = Decoder

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        pixel_values: torch.FloatTensor,
        annotation: Optional[torch.FloatTensor] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SemanticSegmenterOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        outputs = self.segformer(
            pixel_values,
            output_hidden_states=True,  # we need the intermediate hidden states
            return_dict=return_dict,
        )

        encoder_hidden_states = outputs.hidden_states if return_dict else outputs[1]

        logits = self.decode_head(encoder_hidden_states)

        loss = None
        if annotation is not None:
            loss_fct = torch.nn.MSELoss()
            loss = loss_fct(logits.squeeze(), annotation.squeeze())

        if not return_dict:
            if output_hidden_states:
                output = (logits,) + outputs[1:]
            else:
                output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SemanticSegmenterOutput(
            loss=loss,
            logits=logits
        )

In [ ]:
#model = Img2Img.from_pretrained("nvidia/mit-b1")
model = Img2Img.from_pretrained("/content/drive/MyDrive/Poolformer_Segformer/checkpoint-1200")

In [ ]:
from transformers import TrainingArguments
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Poolformer_Segformer",
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    num_train_epochs=100,
    fp16=True,
    save_steps=100,
    eval_steps=20,
    logging_steps=10,
    learning_rate=2e-6,
    save_total_limit=10,
    remove_unused_columns=False,
)

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.state.log_history

###Download

In [ ]:
import os, tarfile

import os
from google.colab import files

def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()

  files.download(output_filename)


make_targz_one_by_one('1224-2-1000-CP_tar', '/content/drive/MyDrive/Poolformer_Segformer/checkpoint-1000')

##OUTPUT

In [ ]:
class Img2ImgDecoder(SegformerDecodeHead):
    def __init__(self, config):
        super().__init__(config)
        self.classifier = torch.nn.Conv2d(config.decoder_hidden_size, 3, kernel_size=1)
        self.RGBnorm = torch.nn.BatchNorm2d(3)

    def forward(self, encoder_hidden_states):
        batch_size = encoder_hidden_states[-1].shape[0]
        h_4, w_4 = encoder_hidden_states[0].size()[2:]

        all_hidden_states = ()
        for encoder_hidden_state, mlp in zip(encoder_hidden_states, self.linear_c):
            if self.config.reshape_last_stage is False and encoder_hidden_state.ndim == 3:
                height = width = int(math.sqrt(encoder_hidden_state.shape[-1]))
                encoder_hidden_state = (
                    encoder_hidden_state.reshape(batch_size, height, width, -1).permute(0, 3, 1, 2).contiguous()
                )

            # unify channel dimension
            height, width = encoder_hidden_state.shape[2], encoder_hidden_state.shape[3]
            encoder_hidden_state = mlp(encoder_hidden_state)
            encoder_hidden_state = encoder_hidden_state.permute(0, 2, 1)
            encoder_hidden_state = encoder_hidden_state.reshape(batch_size, -1, height, width)
            # upsample
            encoder_hidden_state = torch.nn.functional.interpolate(
                encoder_hidden_state, size=(h_4*4, w_4*4), mode="bicubic", align_corners=False
            )
            all_hidden_states += (encoder_hidden_state,)

        hidden_states = self.linear_fuse(torch.cat(all_hidden_states[::-1], dim=1))
        hidden_states = self.batch_norm(hidden_states)
        hidden_states = self.activation(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # logits are of shape (batch_size, RGB, height, width)
        logits = self.classifier(hidden_states)
        logits = self.RGBnorm(logits)
        logits = torch.sigmoid(logits)

        return logits

In [ ]:
Encoder = PoolFormerModel.from_pretrained(""); Encoder.to('cuda:0') #checkpoint
Decoder = Img2ImgDecoder.from_pretrained(""); Decoder.to('cuda:0') #checkpoint
print('Loading from pretrained weights: Done!')

In [ ]:
feature_extractor = PoolFormerFeatureExtractor.from_pretrained("sail/poolformer_s36")

In [ ]:
class Img2Img(SegformerForSemanticSegmentation):
    def __init__(self, config):
        super().__init__(config)
        self.segformer = Encoder
        self.decode_head = Decoder

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        pixel_values: torch.FloatTensor,
        annotation: Optional[torch.FloatTensor] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SemanticSegmenterOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        outputs = self.segformer(
            pixel_values,
            output_hidden_states=True,  # we need the intermediate hidden states
            return_dict=return_dict,
        )

        encoder_hidden_states = outputs.hidden_states if return_dict else outputs[1]

        logits = self.decode_head(encoder_hidden_states)

        loss = None
        if annotation is not None:
            loss_fct = torch.nn.MSELoss()
            loss = loss_fct(logits.squeeze(), annotation.squeeze())

        if not return_dict:
            if output_hidden_states:
                output = (logits,) + outputs[1:]
            else:
                output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SemanticSegmenterOutput(
            loss=loss,
            logits=logits
        )

In [ ]:
model = Img2Img.from_pretrained("/content/drive/MyDrive/Poolformer_Segformer/checkpoint-3300")

#Inference


In [ ]:
from torchvision.transforms import ToPILImage

In [ ]:
image = Image.open('/content/drive/MyDrive/testsets/RNI15/Boy.png')
image

In [ ]:
image = torch.unsqueeze(ToTensor()(image), 0).to('cuda:0')
with torch.no_grad(): outputs = model(image)
ToPILImage()(outputs['logits'][0])

In [ ]:
print(outputs['logits'][0])

In [ ]:
pip install --upgrade git+https://github.com/Lyken17/pytorch-OpCounter.git

In [ ]:
from thop import profile

with torch.cuda.device(0):  # 指定 GPU 編號，如果有的話
    model = Img2Img(Encoder.config)
    flops, params = profile(model, inputs=(torch.randn(1, 3, 256, 256).to('cuda:0'), torch.randn(1, 3, 256, 256).to('cuda:0')))
    print(f"FLOPs: {flops / 1e9:.2f}G, Parameters: {params / 1e6:.2f}M")
